<a href="https://colab.research.google.com/github/DavidSchineis/Math-Physics/blob/main/Copy_of_Lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstract
This lab uses vector math and calculus to visualize vector fields and verify Stokes Theorem. We began by using scipy.interpolate to take line integrals and we tested its accuracy by comparing it with hand calculated integral solutions as well as directly plotting the integral. After this, we moved onto vector fields taking and visualizing the gradient, divergence, and curls in 3D. Then, we used these tools to verify the Stokes Theorem on a square and then on a quarter circle. In both cases the surface integral of the curl matched the circulation near boundary, verifying Stokes Theorem.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy import cos, sin, tan, cosh, sinh, tanh

Last time we have performed numerical integration and differentiation, as well as more complex partial differentiation in order to calculate gradient, divergence, and curl. Throughout this lab, it may be useful to refer back to it on occasion.


2d integrals function similarly to partial derivatives. You simply need to repeat the integration twice, first with respect to x along axis=0. This will reduce it to a 1d array that you can then integrate with respect to y (axis would not need to be specified, since axis=1 no longer exists).

We will first numerically compute a 2d integral of $$f(x,y)=\sqrt{x}+y^2$$ for x from 0 to 1, and for y from -1 to 1. Confirm it through integrating it on paper.

Remember that for meshgrid, indexing should be set to 'ij' to keep the convention of axis=0 being associated with x axis.

In [ ]:
x,y=np.arange(0,1,0.001),np.arange(-1,1,0.001)
xx,yy = np.meshgrid(x,y, indexing='ij')

fxy=np.sqrt(xx) + yy**2

Ix=np.trapezoid(fxy,x,axis=0)
Ixy=np.trapezoid(Ix,y)

print(Ixy)

It's also possible to do arbitrary line inegrals by evaluating the value of the function along the line, and then integrating the resulting array. It is possible to do it two different ways.

If we know the function explicitly - we do not necessarily need a meshgrid.
Create variables x1 from 0 to 0.5 and y1 that follows $y=\sqrt{x}$. Then evauate the same f(x,y) as above, make sure to use a unique variable name for all of them, without reusing existing ones.

Then use pcolormesh to plot fxy above, and plot x1 vs y1 on top of it.

In [ ]:
c=plt.pcolormesh(xx, yy, fxy,vmin=0,vmax=1.5)
plt.colorbar(c,label='f(x,y)')
x1=np.linspace(0,0.5)
y1=np.sqrt(x1)

plt.plot(x1,y1)

fxy1=np.sqrt(x1) + y1**2
plt.show()

### Caption
This plots f(x,y) = sqrt(x) + y^2 as a colormap and y = sqrt(x) overlayed.

Alternatively, when we do not necessarily know the function, but we do have a 2d array of values, we can interpolate across it.

In [ ]:
from scipy.interpolate import RegularGridInterpolator
interp = RegularGridInterpolator((x, y), fxy) #creates a function that associates existing x & y
                                                #grid points with the 2d array off the function
fxy2=interp((x1,y1))

plt.plot(fxy1,fxy2)
plt.xlabel('f(x,y) obtained from evaluating the function')
plt.ylabel('f(x,y) obtained from interpolation')
plt.show()

### Caption
This plots f(x,y) = sqrt(x) + y^2 evaluated by the function (x-axis) versus evaluated by interpolating the 2D grid (y-axis). The result is diagonal, showing agreement between the two evaluations.

When evaluating this line integral, we evaluate it with respect to the evenly spaced x1 that was used to define all of these expressions. Compare integration of both fxy1 and fxy2.

In [ ]:
I1=np.trapezoid(fxy1, x1, axis=0)
I2=np.trapezoid(fxy2, x1, axis=0)

print('Line integral of f(x,y) is', I1,'using direct evaluation along the line')
print('Line integral of f(x,y) is', I2,'using interpolation along the line')

Evaluate the value of the line integral yourself to confirm.

Returning back to the surface integrals, so far we have evaluated it over a rectangular area. Doing arbitrary shaped area integrals is not much harder - you just need to mask out (i.e., set to 0) the value of $f(x,y)$ outside of your area.

In [ ]:
fxy3=fxy.copy()

a=np.where((yy<0) | (xx>0.5) | (np.sqrt(xx)<y))
#the paretheses are important between multiple expressions
fxy3[a]=0

#display the resulting array and integrate over it
c=plt.pcolormesh(xx, yy, fxy3)
plt.colorbar(c,label='f(x,y)')
plt.show()

Ix=np.trapezoid(fxy,x,axis=0)
Ixy=np.trapezoid(Ix,y)
print('area',Ixy)

### Caption
This plots the integral of f(x,y) = sqrt(x) + y^2 as a colormap where we masked out any values outside of f(x,y) to show only the integral values.

## Stokes theorem

Now, using these computational techniques, we can confirm the Stokes theorem numerically.

We will begin with the function $$\vec{v}=(\cos{x}\sinh{y}\tan{z})\hat{x}+(\sin{x}\tanh{y}\cos{z})\hat{y}+(\tan{x}\cosh{y}\sin{z})\hat{z}$$

We will use the surface defined by coordinates of $(1,0,0),(1,1,0),(1,1,1),(1,0,1)$.

Take the curl of the vector, perform an area integral of the resulting curl. Then compare it to the sum of 4 line integrals. Use the plot to ensure self-consistency of the orientation.

In [ ]:
#function to define vector v
def vector(xx,yy,zz):
    vx=cos(xx)*sinh(yy)*tan(zz)
    vy=sin(xx)*tanh(yy)*cos(zz)
    vz=tan(xx)*cosh(yy)*sin(zz)
    return(vx,vy,vz)

#function to define
def curl(x,y,z,vx,vy,vz):
    cx=(np.gradient(vz,y,axis=1) - np.gradient(vy,z,axis=2))
    cy=(np.gradient(vx,z,axis=2) - np.gradient(vz,x,axis=0))
    cz=(np.gradient(vy,x,axis=0) - np.gradient(vx,y,axis=1))
    return(cx,cy,cz)

#function that returns a plane defined by 3 set points in 3d space, a, b, c
def surface(a,b,c,u,v,axis='xy'):
    a,b,c=np.array(a),np.array(b),np.array(c)
    normal = np.cross(b-a,c-a)
    d = np.dot(a,normal)
    if (axis=='xy') | ((normal[0]==0) & (normal[1]==0)):
        xs,ys=np.meshgrid(u,v, indexing='ij')
        zs=(-normal[0] * xs - normal[1] * ys + d) * 1. / normal[2]
    if (axis=='yz') | ((normal[1]==0) & (normal[2]==0)):
        ys,zs=np.meshgrid(u,v, indexing='ij')
        xs=(-normal[2] * zs - normal[1] * ys + d) * 1. / normal[0]
    if (axis=='xz') | ((normal[0]==0) & (normal[2]==0)):
        xs,zs=np.meshgrid(u,v, indexing='ij')
        ys=(-normal[0] * xs - normal[2] * zs + d) * 1. / normal[1]

    normal=normal/np.sqrt(np.sum(normal**2))
    return normal,xs,ys,zs

#interpolates 3d vectors along a given 2d surface. Also, 2d surface along 1d line
def interpNd(axes,vector,value):
    interp_cxs = RegularGridInterpolator(axes, vector[0])
    interp_cys = RegularGridInterpolator(axes, vector[1])
    interp_czs = RegularGridInterpolator(axes, vector[2])
    x=interp_cxs(value)
    y=interp_cys(value)
    z=interp_czs(value)
    return x,y,z


#plot of the result
def makeplot(xx,yy,zz,xs,ys,zs,cxs,cys,czs):
    fig = go.Figure()

    fig.add_trace(go.Isosurface(
        x=xx.flatten(),
        y=yy.flatten(),
        z=zz.flatten(),
        value=(xx*0).flatten(),
        opacity=0.3,
        showscale=False
        ))
    fig.add_trace(go.Surface(
        x=xs,
        y=ys,
        z=zs,
        showscale=False
        ))


    fig.add_trace(go.Cone(
        x=xs.flatten(),
        y=ys.flatten(),
        z=zs.flatten(),
        u=cxs.flatten(),
        v=cys.flatten(),
        w=czs.flatten(),
        colorscale='bluered_r',
        cmin=0,
        cmax=0.0001,
        name='curl v(x,y,z)',
        sizeref=1,
        showlegend=False,
        showscale=False))
    return fig

x,y,z=np.linspace(0,1,20),np.linspace(0,1,20),np.linspace(0,1,20) #defines the axes
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij') #creates a mesh of points
vx,vy,vz=vector(xx,yy,zz) #creates vector v
cx,cy,cz=curl(x,y,z,vx,vy,vz) # takes curl of vector v


u=np.linspace(0,1,20)
v=np.linspace(0,1,30)
normal,xs,ys,zs=surface((1,0,0),(1,1,0),(1,0,1),u,v,axis='yz') #defines a surface from 3 points; u & v are used to reference any points on this surface
#normal is a vector that is perpendicular to this surface;
#normal[0] is the x hat component, normal[1] is the y hat component, normal[2] is the z hat component

vxs,vys,vzs=vector(xs,ys,zs) #vector along the surface
print(normal)
cxs,cys,czs=interpNd((x,y,z),(cx,cy,cz),(xs,ys,zs)) #interpolates curl vector to a given surface

#take a dot product of normal vector and the curl interpolated along the surface
dot=(normal[0]*cxs + normal[1]*cys + normal[2]*czs)

#2d integral of dot, first with respect to u along axis=0, and with respect to v
Ia = np.trapezoid(dot, u, axis=0)
area = np.trapezoid(Ia, v, axis=0)
print('Area integral of the curl is',np.round(area,5))

#Interpolating vector v onto the line defined by x1=1,y1=y,z1=0
x1=np.ones(20)
y1=np.linspace(0,1,20)
z1=np.zeros(20)
ix,iy,iz=interpNd((u,v),(vxs,vys,vzs),(y1,z1)) #our plane is defined in the yz plane
a=(np.trapezoid(ix,x1)+np.trapezoid(iy,y1)+np.trapezoid(iz,z1))










#find other 3 line integrals to fully enclose the surface; ensure that you are going in a consistent direction around the perimeter
#(i.e, sometimes you need to define linspace of your coordinates to be from 0 to 1, and sometimes from 1 to 0)


yb = np.ones(20)
zb = np.linspace(0,1,20)
ix,iy,iz = interpNd((u,v),(vxs,vys,vzs),(yb,zb))
b = np.trapezoid(ix, np.ones_like(zb)) + np.trapezoid(iy, yb) + np.trapezoid(iz, zb)



yc = np.linspace(1,0,20)
zc = np.ones(20)
ix,iy,iz = interpNd((u,v),(vxs,vys,vzs),(yc,zc))
c = np.trapezoid(ix, np.ones_like(yc)) + np.trapezoid(iy, yc) + np.trapezoid(iz, zc)


yd = np.zeros(20)
zd = np.linspace(1,0,20)
ix,iy,iz = interpNd((u,v),(vxs,vys,vzs),(yd,zd))
d = np.trapezoid(ix, np.ones_like(zd)) + np.trapezoid(iy, yd) + np.trapezoid(iz, zd)



#sum these 4 line integrals together to confirm Stokes theorem. Make sure you have correct direction for the lines
print('Line integral of the vector is',np.round(a+b+c+d,5))

#use this to help you orient yourself in the 3d space to define your line integrals
fig=makeplot(xx,yy,zz,xs,ys,zs,cxs,cys,czs)
fig.add_trace(go.Scatter3d(
        x=x1,
        y=y1,
        z=z1))
fig.show()

### Caption
This plots Stokes Theorem on 𝑣⃗=(cos𝑥sinh𝑦tan𝑧)𝑥̂+(sin𝑥tanh𝑦cos𝑧)𝑦̂+(tan𝑥cosh𝑦sin𝑧)𝑧̂. The colormapped surface represents the x=1 plane while the cones are the curl field being applied.

## Now repeat this, using the same function and in the same plane, but across an area defined by a quarter-circle centerd at 0, satisfying $y^2+z^2<1$

In [ ]:
cxs,cys,czs=interpNd((x,y,z),(cx,cy,cz),(xs,ys,zs)) #interpolates curl vector to a given surface


a=np.where((ys**2 + zs**2) > 1) # conditional statement that depends on ys and zs (not y & z!)
             # to select the area _outside_ the circle (i.e., inverse of the stated criteria that we want to keep);
             # we will use it to mask out an area of the square surface along which curl is defined
             # that does not satisfy our condition
cxs[a]=0
cys[a]=0
czs[a]=0

dot=(normal[0]*cxs + normal[1]*cys + normal[2]*czs)
Ia = np.trapezoid(dot, u, axis=0)
area = np.trapezoid(Ia, v)
print('Area integral of the curl is',np.round(area,5))


theta=np.linspace(0,np.pi/2,20)
#find 3 line integrals to fully enclose the resulting surface.
#One of these lines will need to depend on angle theta provided above to appropriately define the quarter circle

y_arc = cos(theta)
z_arc = sin(theta)
ix,iy,iz = interpNd((u,v),(vxs,vys,vzs),(y_arc, z_arc))
a = np.trapezoid( iy * (-np.sin(theta)) + iz * (np.cos(theta)), theta )

z_rad = np.linspace(1,0,20)
y_rad = np.zeros(20)
ix,iy,iz = interpNd((u,v),(vxs,vys,vzs),(y_rad, z_rad))
b = np.trapezoid(iz, z_rad)

y_rad = np.linspace(0,1,20)
z_rad = np.zeros(20)
ix,iy,iz = interpNd((u,v),(vxs,vys,vzs),(y_rad, z_rad))
c = np.trapezoid(iy, y_rad)

print('Line integral of the vector is',np.round(a+b+c,5))

fig=makeplot(xx,yy,zz,xs,ys,zs,cxs,cys,czs)

fig.add_trace(go.Scatter3d(
        x=np.ones(20),
        y=np.cos(np.linspace(0,np.pi/2,20)),
        z=np.sin(np.linspace(0,np.pi/2,20))))

fig.show()

### Caption
This plots Stokes Theorem on 𝑣⃗=(cos𝑥sinh𝑦tan𝑧)𝑥̂+(sin𝑥tanh𝑦cos𝑧)𝑦̂+(tan𝑥cosh𝑦sin𝑧)𝑧̂. The colormapped surface represents the x=1 plane which includes a quarter circle while the cones are the curl field being applied.

## Extra credit

A vector field is given in cyclindrical coordinates by $$\vec{v}=s^2\hat{\phi}$$
Verify Stokes theorem using a shape given by a circle with radius 1 on xy plane

In [ ]:
#def vector(xx,yy,zz):
#    s= #depends on the cartesian coordinates
#    phi=np.arctan2(yy,xx)
#    vx=#
#    vy=#
#    vz=#
#    return(vx,vy,vz)